## Building embeddings from the emotion data

In [2]:
import pandas as pd
import nltk

filepath = '../lab3.machine_learning/data/MELD/train_sent_emo.csv'
dftrain = pd.read_csv(filepath)
### The data has some problematic strings with encoding problems. The next code removes some of these from the utterances
# Try to fix encoding
dftrain['Utterance'] = dftrain['Utterance'].str.replace("\x92|\x97|\x91|\x93|\x94|\x85", "'")

filepath = '../lab3.machine_learning/data/MELD/test_sent_emo.csv'
dftest = pd.read_csv(filepath)
dftest['Utterance'] = dftest['Utterance'].str.replace("\x92|\x97|\x91|\x93|\x94|\x85", "'")

/Users/piek/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/piek/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':


In [8]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

#Function to remove punctuation from word tokens, 
#Takes a list of tokens as input

#Note that these functions only work if you also imported NLTK and string before calling the function
def remove_punct(tokens):
    # punct is a string with all punctuation tokens: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    punct = string.punctuation
    # empty list in which we put the clean tokens
    tokens_clean = []

    # Iterate over all characters in tokens 
    # and only keeps them if not in punct
    for t in tokens:
        if t not in punct:
            tokens_clean.append(t)
            
    return tokens_clean
# If you want to process other text than the Leipzig corpus that is not split into sentences,
# you can call the next function. The difference is:
# - we read the complete file as a text string
# - we apply the NLTK sent_tokenize function to the get a list of sentences
# - we do not need to remove the identifier
def preprocess_rawtext(texts):
    clean_sentences = []
    for text in texts: 
        sentences = sent_tokenize(text.strip())

        for sentence in sentences:
            tokens = word_tokenize(sentence.lower())
            tokens_clean = remove_punct(tokens)
            clean_sentences.append(tokens_clean)
    return clean_sentences

In [9]:
### the first loop gets the utterances
training_instances = dftrain['Utterance']
### the first loop gets the utterances
test_instances = dftest['Utterance']

clean_sentences = preprocess_rawtext(training_instances)
clean_sentences.extend(preprocess_rawtext(test_instances))

In [12]:
print('Number of sentences=',len(clean_sentences))
#we print a few sentences to see how it looks like
print(clean_sentences[201:208])

Number of sentences= 17757
[['ross', 'can', 'i', 'talk', 'to', 'you', 'for', 'a', 'minute'], ['yes', 'please'], ['so', 'what', "'s", 'going', 'on'], ['uh', 'well', '...', 'joey', 'and', 'i', 'broke', 'up'], ['oh', 'my', 'god', 'wh-what', 'happened'], ['joey', 'is', 'a', 'great', 'guy', 'but', 'we', "'re", 'just', '...', 'so', 'different'], ['i', 'mean', 'during', 'your', 'speech', 'he', 'kept', 'laughing', 'at', 'homo', 'erectus']]


In [22]:
filepath = '../lab3.machine_learning/data/wassa/training/all.train.tsv'
dftweets_train = pd.read_csv(filepath, sep='\t')

In [23]:
tweet_instances = dftweets_train['Tweet']

In [24]:
clean_sentences.extend(preprocess_rawtext(tweet_instances))
print('Number of sentences=',len(clean_sentences))

Number of sentences= 23770


In [43]:
filepath = '../lab3.machine_learning/data/smile/datasets-773597-1332516-smile-annotations-final.csv'
smile = pd.read_csv(filepath, sep=',')

In [44]:
smile_instances = dftweets_train['Tweet']
clean_sentences.extend(preprocess_rawtext(tweet_instances))
print('Number of sentences=',len(clean_sentences))

Number of sentences= 29783


In [48]:
# You need to do the next commands only once. When you have succesfully created and saved the embeddings you can load them afterwards
from gensim.models import Word2Vec

w2v = Word2Vec(clean_sentences, size=30, window =4, min_count =1)

In [49]:
print('Vector size =', w2v.vector_size)

Vector size = 30


In [50]:
king_vector = w2v["guy"]
print(len(king_vector))
print(king_vector)

30
[ 0.18709244  0.6194727   0.74906373 -0.91969603 -0.7626541  -0.6097418
 -0.5105595  -0.19118078 -0.12841226  0.6019333   1.7183944  -0.34294078
 -0.144883   -1.1201503  -0.41258347 -0.52747124  0.1678897   0.72965366
 -0.5247881  -1.6217178   0.09158331 -1.3900898   1.0378044  -1.5165662
 -0.526757   -0.3254857  -0.6291967   0.3353514  -1.3505806  -0.58544546]


/Users/piek/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [53]:
w2v.similar_by_word("boy", topn=10)

/Users/piek/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('lol', 0.9983160495758057),
 ('negative', 0.9983031749725342),
 ('thanks', 0.9979099035263062),
 ('v', 0.9977244734764099),
 ('indian', 0.9976120591163635),
 ('mr.', 0.9974886775016785),
 ('hello', 0.9970127940177917),
 ('haha', 0.996992290019989),
 ('action', 0.9967154860496521),
 ("y'all", 0.9965254068374634)]